# Exploration 1

## (미니프로제트) 가위,바위,보 분류기 

평가문항 및 상세기준
1. 이미지 분류기 모델이 성공적으로 만들어졌는가?
   - LMS의 강의 내용에 따라 image 분류기는 성공적으로 와료 되었음.
   - 트레이닝이 정상적으로 수행되었음
   - 하이퍼파라미터들의 특별한 변경은 없었음.=> 변경하여 테슽 해봐도 특별한 향상이 보이지 않음.

2. 오버피팅을 극복하기 위한 적절한 시도가 있었는가?
   - 데이터셋의 다양성, 정규화 등의 시도가 적절하였음
   - 초기에 주어진 이미지(학습용300개, 테스트용300개)로 작업했을때는 테스트결과는 60%이하의 값을 보임.
   - dataset을 교체함. => 직접 사진을 찍은 이미지로 dataset을 변경한 후 테스트결과 73%의 정확도를 보임.
   - 학습용데이터와 테스트용데이터의 비율이 1:1로 한 것과 주어진 이미지의 품질이 
     첫 dataset을 가지고 한 작업의 낮은 정확도의 원인으로 생각됨
   - 두번째 작업은 직접찍은 이미지를 사용하고, 학습과 테스트에 사용한 이미지의 비율을 8:2로 하였음.
   
   
3. 분류모델의 test accuracy가 기준 이상 높게 나왔는가?
   - 60% 이상 도달하였습니다. => 73%(test_accuracy: 0.7333333492279053)
   

In [1]:
##################################################################################
# PIL 라이브러리 import 하기

from PIL import Image
import glob
import os

print("PIL 라이브러리 import 완료!\n")


##################################################################################
# 이미지를 불러와서 28x28 사이즈로 변경하는 함수 resize_images()
# 사이즈 변경 하기(함수를 호출한다)

def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
#	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS) # 높은 해상도의 사진(영상)을 낮은 해상도로 변환 할때 깨진 패턴의 형태로 
                                                            # 나타나게 되는 것을 최소화 시켜주는 방법을 안티엘리어싱

		new_img.save(img, "JPEG")
    
#	print(len(images), " images resized.")
	
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
resize_images(image_dir_path)

#print("가위 이미지 resize 완료!\n")

# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock"
resize_images(image_dir_path)

#print("바위 이미지 resize 완료!\n")


# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper"
resize_images(image_dir_path)

#print("보 이미지 resize 완료!\n")


##################################################################################
# 가위, 바위, 보 데이터를 읽을 수 있는 load_data() 함수
# 가위: 0, 바위: 1, 보: 2

import numpy as np

def load_data(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3 # ?????? 뭐지?????? RGB인가??? => RGB이군!
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs = np.zeros(number_of_data*img_size*img_size*color, dtype=np.int32).reshape(number_of_data, img_size, img_size, color)
    labels = np.zeros(number_of_data, dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path + '/scissor/*.jpg'): # 해당 디렉토리의 이미지 개수 만클 for문이 도는 것 인가 ?????????? 
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx, : , : , : ] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 0   # 가위 : 0
        idx = idx + 1

    for file in glob.iglob(img_path + '/rock/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx, : , : , : ]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 1   # 바위 : 1
        idx = idx + 1  
    
    for file in glob.iglob(img_path + '/paper/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx, : , : , : ]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 2   # 보 : 2
        idx = idx + 1
        
#    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_train, y_train) = load_data(image_dir_path, 270)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화


#print("x_train shape: {}".format(x_train.shape))
#print("y_train shape: {}".format(y_train.shape))
#print("")

##################################################################################
# 데이터의 준비 완료!!! 딥러닝 네트워크를 설계 시작 !!
 
import tensorflow as tf
from tensorflow import keras
import numpy as np

# model을 직접 만들어 보세요.
# Hint! model의 입력/출력부에 특히 유의해 주세요. 가위바위보 데이터셋은 MNIST 데이터셋과 어떤 점이 달라졌나요?
# [[YOUR CODE]]

#####################################
# 바꿔 볼 수 있는 하이퍼파라미터들
#####################################
n_channel_1=16
n_channel_2=64
n_dense=32
n_train_epoch=10

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

#print('Model에 추가된 Layer 개수: ', len(model.layers))

#model.summary()

##################################################################################
# 딥러닝 네트워크 설계 완료 !! 학습 시작 !!!

# model을 학습시키는 코드를 직접 작성해 보세요.
# Hint! model.compile()과 model.fit()을 사용해 봅시다.
# [[YOUR CODE]]

x_train_reshaped = x_train_norm.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.
#x_test_reshaped = x_test_norm.reshape( -1, 28, 28, 3)



model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])


#print("x_train_reshaped :", x_train_reshaped)
#print("y_train :", y_train)
model.fit(x_train_reshaped, y_train, epochs = n_train_epoch)

##################################################################################
# for 테스트 이미지 작업
# 이미지를 불러와서 28x28 사이즈로 변경하는 함수 resize_images()
# 사이즈 변경 하기(함수를 호출한다)

# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서 사이즈 변경
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/scissor"
resize_images(image_dir_path)

#print("가위 테스트 이미지 resize 완료!\n")

# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서 사이즈 변경
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/rock"
resize_images(image_dir_path)

#print("바위 테스트 이미지 resize 완료!\n")


# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서 사이즈변경
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/paper"
resize_images(image_dir_path)

#print("보 테스트 이미지 resize 완료!\n")


##################################################################################
# 가위, 바위, 보 데이터를 읽을 수 있는 load_data() 함수
# 가위: 0, 바위: 1, 보: 2

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"
(x_test, y_test) = load_data(image_dir_path, 30)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

#print("x_test shape: {}".format(x_test.shape))
#print("y_test shape: {}".format(y_test.shape))
#print("")


##################################################################################
# 가위, 바위, 보 데이터를 읽을 수 있는 load_data() 함수
# 가위: 0, 바위: 1, 보: 2


x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화
x_test_reshaped = x_test_norm.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.

test_loss, test_accuracy = model.evaluate(x_test_reshaped,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))



PIL 라이브러리 import 완료!

Epoch 1/10
9/9 [==============================] - 9s 18ms/step - loss: 1.0577 - accuracy: 0.4407
Epoch 2/10
9/9 [==============================] - 0s 3ms/step - loss: 0.9012 - accuracy: 0.6815
Epoch 3/10
9/9 [==============================] - 0s 3ms/step - loss: 0.6701 - accuracy: 0.7111
Epoch 4/10
9/9 [==============================] - 0s 3ms/step - loss: 0.5330 - accuracy: 0.7778
Epoch 5/10
9/9 [==============================] - 0s 3ms/step - loss: 0.4916 - accuracy: 0.6926
Epoch 6/10
9/9 [==============================] - 0s 3ms/step - loss: 0.4625 - accuracy: 0.7037
Epoch 7/10
9/9 [==============================] - 0s 3ms/step - loss: 0.4150 - accuracy: 0.7963
Epoch 8/10
9/9 [==============================] - 0s 3ms/step - loss: 0.3903 - accuracy: 0.8481
Epoch 9/10
9/9 [==============================] - 0s 3ms/step - loss: 0.3616 - accuracy: 0.8593
Epoch 10/10
9/9 [==============================] - 0s 3ms/step - loss: 0.3547 - accuracy: 0.8556
1/1 - 0s - loss: